In [38]:
from py2neo import Graph

graph = Graph("http://localhost:7474/db/default.graphdb/", username="neo4j", password="abc")
graph.delete_all()

In [39]:
graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///users.csv" AS line
CREATE (:User { id: line.id, screen_name: line.screen_name, location: line.location })
""")

graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///tweets.csv" AS line
CREATE (:Tweet { id: line.id, user_id: line.user_id, text: line.text })
""")

graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///retweets.csv" AS line
CREATE (:Retweet { tweet_id: line.tweet_id, user_id: line.user_id })
""")

graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///tags.csv" AS line
CREATE (:Tag { label: line.label })
""")

graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///tweet_tags.csv" AS line
CREATE (:TweetTag { tweet_id: line.tweet_id, label: line.label })
""")

graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///personalities.csv" AS line
CREATE (:Personality { user_id: line.user_id, similar_id: line.similar_personalities, different_id: line.different_personalities })
""")
                     
graph.evaluate("""
LOAD CSV WITH HEADERS FROM "file:///locations.csv" AS line
CREATE (:Location { sub_state: line.sub_state })
""")


In [40]:
graph.evaluate("""
MATCH (a:User), (b:Tweet)
WHERE a.id = b.user_id
CREATE (a)-[r:TWEETED]->(b)
""")
         
graph.evaluate("""
MATCH (a:Tweet), (b:User), (c:Retweet)
WHERE a.id = c.tweet_id AND b.id = c.user_id
CREATE (b)-[r:RETWEETED]->(a)
""")
         
graph.evaluate("""
MATCH (a:Tweet), (b:Tag), (c:TweetTag)
WHERE a.id = c.tweet_id AND c.label = b.label
CREATE (a)-[r:TAGGED]->(b)
""")

graph.evaluate("""
MATCH (a:User), (b:User), (c:Personality)
WHERE a.id = c.user_id AND b.id = c.similar_id
CREATE (a)-[r:SIMILAR]->(b)
""")

graph.evaluate("""
MATCH (a:User), (b:User), (c:Personality)
WHERE a.id = c.user_id AND b.id = c.different_id
CREATE (a)-[r:DIFFERENT]->(b)
""")
         
graph.evaluate("""
MATCH (a:User), (b:Location)
WHERE a.location = b.sub_state
CREATE (a)-[r:LIVES]->(b)
""")